<a href="https://colab.research.google.com/github/senaruna0112/yahoonews/blob/main/yahoo_NCI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# webスクレイピング

## Colab上でWeb Driverを入れてスクレイピングできる環境を作る

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 963 kB 14.5 MB/s 
     |████████████████████████████████| 138 kB 66.3 MB/s 
     |████████████████████████████████| 359 kB 85.0 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 3.6 MB 72.3 MB/s 
     |████████████████████████████████| 58 kB 5.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu

# yahooニュースのスクレイピング

In [5]:
# import libraries
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import pandas as pd

# initialize chrome driver (settings)
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# create webdriver
driver = webdriver.Chrome('chromedriver',options=options)

In [6]:
MAX_PAGE = 4
href_list = []

for i in range(1, MAX_PAGE+1):
  print(f"https://news.yahoo.co.jp/topics/business?page={i}")
  driver.get(f"https://news.yahoo.co.jp/topics/business?page={i}")
  # html = driver.page_source.encode('utf-8')
  html = driver.page_source

  # bs4の設定
  soup = BeautifulSoup(html, 'html.parser')
#   print(soup)

  # 現在いるページにある記事のリンク一覧
  tmp = soup.find_all(class_="newsFeed_item_link")
  print(tmp)
  # linkのtextをとってくる
  for tar in tmp:
    href_list.append(tar.get("href"))
  time.sleep(1)

https://news.yahoo.co.jp/topics/business?page=1
[<a class="newsFeed_item_link" data-cl-params="_cl_vmodule:st_topics;_cl_link:title;_cl_position:1;" data-rapid_p="46" data-ylk="rsec:st_topics;slk:title;pos:1;" href="https://news.yahoo.co.jp/pickup/6419216"><div class="newsFeed_item_thumbnail"><div class="sc-dVhcbM jKVCqr thumbnail thumbnail-small"><img alt="" class="sc-eqIVtm koAuUY" loading="lazy" src="https://news-pctr.c.yimg.jp/t/news-topics/images/tpc/2022/2/24/2ec81f3fdab73f168d4bcb696618ba2045071277e4ce1221de91b82193129d7c.jpg"/></div></div><div class="newsFeed_item_text"><div class="newsFeed_item_title">松野氏 原油やガス大きな懸念ない</div><div class="newsFeed_item_sub"><div class="newsFeed_item_sourceWrap"><time class="newsFeed_item_date">2/24(木) 18:16</time></div></div></div></a>, <a class="newsFeed_item_link" data-cl-params="_cl_vmodule:st_topics;_cl_link:title;_cl_position:2;" data-rapid_p="47" data-ylk="rsec:st_topics;slk:title;pos:2;" href="https://news.yahoo.co.jp/pickup/6419214"><div c

In [7]:
titles, contents = [], []
for target_url in href_list:
  driver.get(target_url)
  title = driver.title
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  url_all = soup.find_all(class_="sc-lkYwMk cIUcbi")[0].find_all(class_="sc-iLWYPX gTRgYH")[0].get("href")
  print(url_all)
#   url_all = soup.find(class_="sc-bQEgQf dXgvwT").get("href")
#   url_all = soup.find_all(class_="sc-bQEgQf dXgvwT")[0].find_all(class_="sc-iScGEx PmDTp")[0].get("href") #井上さんが書いた。　classが変更になっている？
  # 全文
  driver.get(url_all)  # 全文があるリンクにアクセス
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  title = soup.find('title').text  # タイトル取得
  text = soup.find(class_="article_body highLightSearchTarget")  # 本文を取得
  if text is None: 
    print(f"取得できませんでした: {target_url}")
    continue
  text = text.get_text()
  print(title)
  titles.append(title)
  contents.append(text)
  time.sleep(1)

https://news.yahoo.co.jp/articles/7d92ac74e34e5a86300ad8adec6fc59a8cb814e4
地政学リスクの高まり、日本経済への影響注視＝官房長官（ロイター） - Yahoo!ニュース
https://news.yahoo.co.jp/articles/51fcb60bcf295e8c2d0f94bdf8f8f1c9b29d7b0c
モスクワ証券取引所、前日終値から５０％値下がり…ルーブルは対ドルで急落（読売新聞オンライン） - Yahoo!ニュース
https://news.yahoo.co.jp/articles/82da87c198fb1064ddbe60319896bbaa1b175782
ウクライナ緊迫化受け東証続落　金の小売価格は最高値更新（産経新聞） - Yahoo!ニュース
https://news.yahoo.co.jp/articles/f3368d248f22125e287914be6df8d70e0456c4c6
昨年の新築マンション発売、３年ぶり増加…コロナ禍で住宅人気高まる（読売新聞オンライン） - Yahoo!ニュース
https://news.yahoo.co.jp/articles/fd841084bd247e980c939a86600ff6554fd84652
【速報】日経平均終値　478円79銭安の2万5970円82銭　 緊迫するウクライナ情勢で売り注文集まり昨年来安値を更新（TBS系（JNN）） - Yahoo!ニュース
https://news.yahoo.co.jp/articles/a242290562cc329c58f0de1fb05cd5203ff5cad0
モスクワ取引所、全ての市場の取引中止（Bloomberg） - Yahoo!ニュース
https://news.yahoo.co.jp/articles/9f45fa7440dcc5c51ddaf5d251ae737b2f61c8de
原油価格が一時１００ドル台に上昇、７年５か月ぶり…ロシアの軍事作戦表明で（読売新聞オンライン） - Yahoo!ニュース
https://news.yahoo.co.jp/articles/4829d4d004985ecf4613885142cb4147745c8f78
【速

In [8]:
df = pd.DataFrame([[title, content] for title, content in zip(titles, contents)], columns=["title", "text"])
df.to_csv("yahoo_news_econ.csv",encoding='utf_8_sig')

# 実装してみる

In [5]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Profmike/yahoo_news_econ2.csv", sep=",")

In [3]:
df.drop(df.columns[0], axis=1)

,title,text
0,なぜタイヤ脱落『9割以上が左後輪』なのか…専門家が指摘する“日本の道路事情”と“ナットの締め...,国土交通省が公開している実験の映像 岐阜県中津川市の中央道で1月、走行中のダンプカーからタイ...
1,新しい客、獲得できるチャンスなのに… 「取材拒否」の飲食店がある理由（オトナンサー） - Y...,なぜ取材拒否？ 飲食店の中には、テレビや雑誌の取材を一切受けない、いわゆる「取材拒否の店」が...
2,【速報】「まん延防止措置」追加適用で“夜の人出”減る 札幌36.5％ 大阪18.6％減（FN...,FNNプライムオンライン27日から追加で「まん延防止等重点措置」が適用された地域では、28日...
3,飲み会後のナイト需要消失が打撃 カラオケ市場、コロナ前から6割減へ 「歌わないカラオケ」で回...,「まん延防止」などで再び遠のく客足 当面は苦しい状況続くまん延防止などで居酒屋客など「ナイト...
4,生乳廃棄問題、給食ない春休みに再燃の可能性も 業界団体が見通し（朝日新聞デジタル） - Ya...,牛乳の消費拡大をPRする金子原二郎農林水産相（中央）ら=2021年12月17日、東京都千代田...
...,...,...
80,しょうゆやジャム…値上げ春先以降も 家計への影響懸念（産経新聞） - Yahoo!ニュース,消費者物価指数が上昇。値上げされる商品など。調理油類も値上げに＝２１日、大阪市城東区のスーパ...
81,ガソリンスタンドの屋根はなぜ平らなのか？ 雪が降ったらどうしてる？（くるまのニュース） - ...,「平ら」は合理性ゆえ ガソリンスタンドを覆う屋根（上屋）の多くは、水平に造られています。なぜ...
82,歌会はなぜ楽しい アイドルやホストも参入、コロナ禍の短歌ブーム（産経新聞） - Yahoo!...,女性アイドル４人らが出演した「アイドル歌会＠詠い納め」。「無記名でも誰の短歌か分かる」と好評...
83,「まさかマイナス？」8歳の貯金箱 駆け込みでATM故障も（時事通信） - Yahoo!ニュース,大量の小銭の行方は… ゆうちょ銀行が2022年1月17日に「硬貨取扱料金」を新設し、硬貨を預...


In [4]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-l

In [6]:
import MeCab
mecab = MeCab.Tagger()
import re
from collections import Counter

##テキストをリスト化する

In [7]:
def dftotext(df):
    texts=[]
    for i in range(len(df)):
        texts.append(df["text"][i])
    return(texts)

In [ ]:
texts=dftotext(df)
texts

In [ ]:
!pip install neologdn
import neologdn
import re

全角・半角の統一と重ね表現の除去 (neologdn)

In [10]:
def normal_text(texts):
    normalized_text=[]
    for i in range(len(texts)):
        normalized_text.append(neologdn.normalize(texts[i]))
    normalized_text
    texts=[]
    for i in range(len(normalized_text)):    
        text = re.sub(r'(\d)([,.])(\d+)', r'\1\3', normalized_text[i]) #桁区切りの数字の除去
        text = re.sub(r'\d+', '0', text) #数字を0にする
        text = tmp = re.sub(r'[!-/:-@[-`{-~]', r' ', text)# 半角記号の置換
        text = text_removed_symbol = re.sub(u'[■-♯]', ' ', text)    # 全角記号の置換 (ここでは0x25A0 - 0x266Fのブロックのみを除去)
        texts.append(text)
    return(texts)

In [11]:
texts=normal_text(texts)

In [13]:
#似たようなニュースを摘出する
num=[11,15,16,17,26,28,29,38,46,54,59,63,64]
texts2 =[]
for i in range(len(df)):
    if i in num:
        texts2.append(df["text"][i])

In [14]:
texts2=normal_text(texts2)

テキストの単語ごとの多次元リストの作成する関数

In [15]:
def get_keywords_list(texts):
    keywords = [[] for i in range(len(texts))]
    for i in range(len(texts)):
        tokenizer = MeCab.Tagger("-Ochasen")
        tokenizer.parse("")
        node = tokenizer.parseToNode(texts[i])
        while node:
            if node.feature.split(",")[0] == u"名詞":
                keywords[i].append(node.surface)
            elif node.feature.split(",")[0] == u"形容詞":
                keywords[i].append(node.feature.split(",")[6])
            elif node.feature.split(",")[0] == u"動詞":
                keywords[i].append(node.feature.split(",")[6])
            node = node.next
    return(keywords)

In [ ]:
keywords=get_keywords_list(texts)
keywords

## ワードのカウント

In [197]:
from collections import defaultdict

In [198]:
def word_freqs(keywords):
    word_freqs =[]
    for i in range(len(keywords)):
        word_freqs.append(defaultdict(int))
        for keyword in keywords[i]:
            word_freqs[i][keyword] += 1
    return(word_freqs)

In [199]:
word_freqs = word_freqs(keywords)
word_freqs

[defaultdict(int,
             {'0': 17,
              'One': 6,
              'kg': 1,
              'km': 1,
              'ある': 4,
              'いう': 2,
              'いる': 7,
              'うち': 1,
              'くる': 1,
              'こと': 3,
              'さん': 1,
              'する': 13,
              'その後': 1,
              'てる': 1,
              'できる': 1,
              'なじみ': 3,
              'なる': 4,
              'の': 3,
              'ぶつかる': 4,
              'みる': 2,
              'やすい': 2,
              'よう': 1,
              'よる': 1,
              'られる': 2,
              'ろっ骨': 1,
              'わかる': 1,
              'ウェブ': 1,
              'エリア': 2,
              'ケガ': 1,
              'サビ': 1,
              'スキマ': 1,
              'スタッドレスタイヤ': 1,
              'タイヤ': 11,
              'ダミー': 1,
              'ダンプカー': 2,
              'トラック': 2,
              'ドライバー': 1,
              'ナット': 1,
              'ナンバー': 1,
              'ニュース': 6,
              'ネジ': 2,
   

In [200]:
from sklearn.feature_extraction import DictVectorizer
V = DictVectorizer()
counts = [[] for i in range(len(texts))]
for i in range(len(texts)):
    counts[i] = V.fit_transform(word_freqs[i]).toarray() 

In [201]:
counts # 次元数が全然違うので前処理がない

[array([[17.,  6.,  1.,  1.,  4.,  2.,  7.,  1.,  1.,  3.,  1., 13.,  1.,
          1.,  1.,  3.,  4.,  3.,  4.,  2.,  2.,  1.,  1.,  2.,  1.,  1.,
          1.,  2.,  1.,  1.,  1.,  1., 11.,  1.,  2.,  2.,  1.,  1.,  1.,
          6.,  2.,  2.,  1.,  1.,  1.,  2.,  3.,  3.,  2.,  2.,  1.,  2.,
          9.,  1.,  1.,  2.,  1.,  3.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  3.,  1.,  2.,  2.,  4.,  2.,  1.,  3.,  1.,  1.,  1.,
          3.,  1.,  1.,  2.,  1.,  1.,  1.,  1.,  1.,  2.,  1.,  3.,  4.,
          2.,  1.,  5.,  1.,  3.,  1.,  1.,  1.,  1.,  3.,  1.,  2.,  1.,
          1.,  1.,  2.,  1.,  1.,  1.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  2.,  1.,  2.,  1.,  4.,  2.,  1.,  1.,  3.,  1.,
          3.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  4.,  1.,  2.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  3.,  1.,  1.,  7.,
          1.,  1.,  1.,  3.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,
          1.]]),
 array([[ 5.,  1., 21

# entitiesの次元を揃えるアプローチ

In [17]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

## bag of wods のdocument-entity Matrix A1 を作成

In [18]:
koyu = []
def split_text_only_nva(text):
    tagger = MeCab.Tagger()

    words = []
    for c in tagger.parse(text).splitlines()[:-1]:
        surface, feature = c.split('\t')
        pos = feature.split(',')[0]
        pos2 = feature.split(',')[1]

        if pos2 == "固有名詞":
            koyu.append(surface)

        if pos == '名詞':
            words.append(surface)
        if pos =="動詞":
            words.append(surface)
        if pos =="形容詞":
            words.append(surface)
    return ' '.join(words)


In [19]:
def get_bow_matrixA(texts):
    messages_list = []
    for i in range(len(texts)):
        messages_list.append(split_text_only_nva(texts[i]))
    messages_list
    docs = np.array(messages_list)
    count = CountVectorizer()
    bags = count.fit_transform(docs)
    matrix=bags.toarray()
    return(matrix)

In [90]:
A1 = get_bow_matrixA(texts)
A1.shape

(85, 4060)

In [91]:
#アプローチ①：取り出したdocumentsのentitiesの次元でmatrixを作成
A12= get_bow_matrixA(texts2)
A12.shape

(13, 498)

In [92]:
#アプローチ⓶　entitiesの次元数は元のmatrixAに合わせて、documentの次元を変更してみる
num=[11,15,16,17,26,28,29,38,46,54,59,63,64]
omit_num=[] 
for i in range(len(texts)):
    if i not in num:
        omit_num.append(i)
A13 = np.delete(A1, omit_num, 0)
A13.shape

(13, 4060)

In [ ]:
set(koyu) # 固有名詞のリスト

##tf-idf のdocument-entity Matrix A2 を作成

In [25]:
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# tf-idf
def get_tf_idf_matrixA(texts):
    messages_list = []
    for i in range(len(texts)):
        messages_list.append(split_text_only_nva(texts[i]))
    messages_list
    docs = np.array(messages_list)
    count = CountVectorizer()
    bags = count.fit_transform(docs)
    tfidf = TfidfTransformer(use_idf=True, norm="l2", smooth_idf=True) #norm='l2'を指定すると単語ベクトルの長さが1になるようコサイン正規化
    np.set_printoptions(precision=2)
    tf_idf = tfidf.fit_transform(bags)
    matrix=tf_idf.toarray()
    return(matrix)

In [26]:
A2=get_tf_idf_matrixA(texts)

In [27]:
#アプローチ①　取り出したdocumentsのentitiesの次元でmatrixを作成
A22=get_tf_idf_matrixA(texts2)

In [28]:
#アプローチ⓶
for i in range(len(texts)):
    if i not in num:
        omit_num.append(i)
A23 = np.delete(A2, omit_num, 0)
A23.shape

(13, 4060)

## NCIを計算する 


In [29]:
from scipy.linalg import svd 
import math
#引数はmatrixAとドキュメントの数
def compute_NCI(x,num_doc):
    U, S, V = svd(x) 
    S = sorted(S, reverse=True)
    print(S)
    k= 5
    sigma = S[:k]
    total =0
    for i in range(len(sigma)):
        a= sigma[i]
        sig2 = a*a
        totalall = total +sig2
        total = totalall
    total
    NCI= math.sqrt(total)
    print(NCI/num_doc)

In [30]:
#全ての文書
compute_NCI(A1,len(A1))

[90.30388127765075, 55.39801234673999, 49.872849118203604, 47.58356966803593, 43.34641687978855, 39.04958898124463, 37.419994015268955, 34.90195555716628, 33.15115819992087, 32.1874022402003, 30.25964521428365, 28.327634310326008, 26.524764806924196, 25.63187905125797, 25.40274143216492, 25.020733618887693, 24.467143515681485, 23.490012396569902, 23.368819163033073, 22.642800733831628, 22.61230583200432, 21.820349498105095, 21.698056428021687, 20.818432016925282, 20.647435890697448, 20.077049895159657, 19.99373323762246, 19.65216928870468, 19.43996773035071, 19.153170780946578, 18.987635761981526, 18.463051831286833, 18.374233416323044, 17.971682321759246, 17.73528770879814, 17.406177385983213, 17.13154754276755, 16.37580988324762, 15.996345573443177, 15.647713262702311, 15.359087282806813, 15.080994667678278, 14.851250762122886, 14.680488881332417, 14.407383314524658, 14.357345188600748, 14.328410266555718, 13.788425827977978, 13.286478941505942, 13.237771277204526, 13.17123540924792,

In [31]:
# アプローチ①類似度が高いニュースを集めて計算（株系のニュース）
compute_NCI(A12,len(A12))# NCIが高い値で出てることから成功していることが分かる 

[34.35059210030014, 26.196276645803223, 17.527656860344216, 14.948159855568056, 11.995268294311007, 10.674129788425383, 9.396855842074357, 8.749635349087669, 6.578166827976631, 5.873337197433001, 4.904073453630953, 4.315882578606782, 2.7248243553260463e-14]
3.8773815260676656


In [32]:
#アプローチ2 次元を揃える
compute_NCI(A13,len(A13))
##ほぼ同じ結果になった。

[34.350592100300155, 26.196276645803213, 17.527656860344212, 14.948159855568043, 11.995268294310973, 10.674129788425386, 9.396855842074359, 8.749635349087665, 6.57816682797663, 5.873337197433001, 4.904073453630955, 4.315882578606782, 2.239008896515585e-14]
3.877381526067665


TF-IDF のNCI

In [34]:
#全ての文書
compute_NCI(A2,len(A2))

[2.34730282701198, 1.9487234487716758, 1.612724283929938, 1.4274410250190093, 1.3493137945336628, 1.3359737317462286, 1.316080140910376, 1.2692790214146263, 1.2343734596070088, 1.2095332449257583, 1.1969402055661085, 1.1364418550874353, 1.1163722667462164, 1.1078207715311426, 1.0984087515023366, 1.088445385510335, 1.0722641119431742, 1.0639212297338672, 1.0568363522597766, 1.043845495164284, 1.0331173483760514, 1.0305021006162014, 1.0259010293110327, 1.021747034292741, 1.0153432152877122, 1.0143308541823313, 1.0118778512910198, 1.0042810162942584, 1.0035216266184517, 0.997138646903482, 0.9941161286309445, 0.9916390190294594, 0.9891137070997303, 0.9849985673753662, 0.9809647436466876, 0.9793613234651781, 0.9719243462913281, 0.9694570936797969, 0.9648447909091616, 0.9631867527909111, 0.9583647907207116, 0.9544336784789701, 0.9515242244257532, 0.9487123209210244, 0.9477036046250742, 0.9419407527619617, 0.9366659703501604, 0.9351768966127418, 0.9278057739066324, 0.924190692553025, 0.921500

In [35]:
#アプローチ①　次元をセレクトニュースに合わせる
compute_NCI(A22,len(A22))


[1.8318947326540052, 1.3367452641478728, 1.0781471646751937, 1.0171648940643387, 0.9864750278134411, 0.9332162668343058, 0.9103859517677404, 0.8810520910425305, 0.8050348913497225, 0.7944928957742441, 0.7200559814335615, 0.6429512837097804, 1.8601192057839638e-16]
0.22178518477874573


In [36]:
#アプローチ2 次元をすべての文書に揃える
compute_NCI(A23,len(A23))
##次元が大きい方が値がlargeなのは直観に反する

[1.9914312954811004, 1.3372582657572034, 1.0418509799687325, 1.0159578550808903, 0.9855011802550843, 0.9152736213337167, 0.8697952680733014, 0.8275410351644917, 0.7681213242143151, 0.7307701023715397, 0.6471966004110418, 0.5788892778643671, 7.11658725162794e-17]
0.22874619249269598


##論文と同じoccurrenceのmatrixを作成する

In [37]:
#paper と同じoccurrence のmatrix
def get_ooccurence_MatrixA(matrix):
    for i in range(len(matrix)):
        for j in range(matrix.shape[1]) :
            if matrix[i][j] >= 1:
                matrix[i][j] = 1
    return(matrix)

In [38]:
A1_occurence= get_ooccurence_MatrixA(A1)
A1_occurence.shape
compute_NCI(A1_occurence,len(A1_occurence))

[32.097428804480344, 22.299832934478633, 17.331745050814632, 16.454730478827717, 15.718663593860569, 15.311727064463897, 14.837315100945634, 13.773525226327184, 13.37010184643109, 13.152811386436733, 12.828879068858097, 12.639074423311875, 12.462708378419105, 12.298860533943351, 12.131076608207566, 12.002818961219262, 11.8380293221747, 11.736042551924156, 11.588053294970994, 11.527073882528681, 11.514056031021152, 11.365213097661783, 11.256168515661821, 11.115824916710602, 10.923214861003947, 10.81485378630182, 10.797293464017258, 10.625272247009065, 10.48004609234755, 10.372827928391606, 10.343560215908528, 10.279572013540733, 10.246057859370687, 10.09364535592256, 10.035267934391511, 9.936947144855443, 9.694602889073666, 9.636578378819705, 9.445846653081993, 9.345262061482876, 9.295518067053433, 9.123189466711763, 9.024837274345968, 8.968697683218538, 8.881028290475466, 8.746779914439218, 8.672890624536773, 8.558361105442833, 8.438426534050365, 8.357606114844899, 8.275611369931314, 8

In [40]:
A12occurrence=get_ooccurence_MatrixA(A12)
A12occurrence.shape
compute_NCI(A12occurrence,len(A12occurrence))

[16.42620631845573, 11.502940212652598, 10.561877036217435, 10.363965415106584, 8.858702258602376, 7.261622835542173, 6.7056658733352394, 6.366332382811838, 5.223384934317868, 5.181547916826923, 4.371550301837981, 3.993811593020474, 1.664178523203997e-14]
2.03458196702614


In [41]:
A13occurrence=get_ooccurence_MatrixA(A13)
A13occurrence.shape
compute_NCI(A13occurrence,len(A13occurrence))
### ほんの少し小さい値になっている

[16.426206318455726, 11.502940212652579, 10.561877036217435, 10.36396541510656, 8.85870225860237, 7.261622835542176, 6.70566587333524, 6.366332382811844, 5.223384934317872, 5.181547916826919, 4.371550301837981, 3.9938115930204723, 9.876053638062149e-15]
2.034581967026138


# トピックモデルの実装

In [42]:
from gensim.corpora import Dictionary
from gensim.models import word2vec
from gensim.models.ldamodel import LdaModel
import gensim

In [43]:
def get_corpus(keywords, max_document_frequency,min_document_frequency):
    dic = Dictionary(keywords)
    # max_document_frequency, min_document_frequency = 5, 2  # 単語の出現頻度の上限下限
    dic.filter_extremes(no_above=max_document_frequency, no_below=min_document_frequency)  # 出現頻度
    corpus = [dic.doc2bow(key) for key in keywords]
    return(corpus)

In [44]:
####　関数内で定義されないので外で変数にしています。　対処したいです。
dic = Dictionary(keywords)
max_document_frequency, min_document_frequency = 5, 2  # 単語の出現頻度の上限下限
dic.filter_extremes(no_above=max_document_frequency, no_below=min_document_frequency)

In [165]:
miscorpus=get_corpus(keywords,5,2)

##頻出ワードがカットできているかの確認

In [166]:
dic.token2id["する"]
# dic.id2token[1430]

9

In [167]:
suru_cnt = 0
suru_doc_freq = 0
for bow in miscorpus:
  flag = False
  for id, cnt in bow:
    if id == dic.token2id["する"]:
      suru_cnt += cnt
      flag = True
  if flag:
    suru_doc_freq += 1
      
suru_cnt, suru_doc_freq

(827, 68)

##コーパスの作成

In [120]:
corpus =get_corpus(keywords,0.25,2)
####　関数内で定義されないので外で変数にしています。　対処したいです。
dic = Dictionary(keywords)
max_document_frequency, min_document_frequency = 0.25, 2  # 単語の出現頻度の上限下限
dic.filter_extremes(no_above=max_document_frequency, no_below=min_document_frequency)

するが消えていることが分かる

In [46]:
# suru_cnt = 0
# suru_doc_freq = 0
# for bow in corpus:
#   flag = False
#   for id, cnt in bow:
#     if id == dic.token2id["する"]:
#       suru_cnt += cnt
#       flag = True
#   if flag:
#     suru_doc_freq += 1
      
# suru_cnt, suru_doc_freq

In [ ]:
corpus #IDと出現回数のbag of wordsになっている。

## LDAモデル化

In [121]:
from gensim.models.ldamodel import LdaModel
lda = LdaModel(corpus = corpus, id2word = dic, num_topics = 3, random_state = 1)

In [125]:
#結果
for t in lda.show_topics(num_words=30):
    print(t)

(0, '0.012*"店" + 0.008*"企業" + 0.005*"会社" + 0.005*"年度" + 0.005*"客" + 0.004*"工場" + 0.004*"池袋" + 0.004*"店舗" + 0.004*"上昇" + 0.004*"部" + 0.003*"影響" + 0.003*"証券" + 0.003*"億" + 0.003*"さん" + 0.003*"市場" + 0.003*"台" + 0.003*"ウイルス" + 0.003*"多い" + 0.003*"・" + 0.003*"株" + 0.003*"停止" + 0.003*"よる" + 0.003*"取材" + 0.003*"生産" + 0.003*"閉店" + 0.003*"前" + 0.003*"拡大" + 0.003*"国内" + 0.003*"上場" + 0.003*"感染"')
(1, '0.012*"店" + 0.007*"会社" + 0.006*"市場" + 0.006*"株" + 0.006*"所" + 0.005*"証券" + 0.005*"銭" + 0.004*"ガソリン" + 0.004*"企業" + 0.004*"・" + 0.004*"取材" + 0.004*"政府" + 0.004*"池袋" + 0.004*"億" + 0.004*"上場" + 0.004*"お金" + 0.004*"上昇" + 0.003*"比" + 0.003*"超える" + 0.003*"米" + 0.003*"公開" + 0.003*"リットル" + 0.003*"抑制" + 0.003*"高" + 0.003*"費" + 0.003*"以上" + 0.003*"投資" + 0.003*"補助" + 0.003*"おる" + 0.003*"取引"')
(2, '0.007*"企業" + 0.007*"事業" + 0.006*"億" + 0.005*"年度" + 0.005*"店" + 0.004*"上昇" + 0.004*"市場" + 0.004*"費" + 0.004*"株" + 0.003*"補助" + 0.003*"高騰" + 0.003*"日本" + 0.003*"グループ" + 0.003*"員" + 0.003*"昨年" + 0.003*"影響" + 0.003*"全国" 

In [124]:
num_words = 5
print("Topic words\n----------------------------------------------------")
# トピックごとの単語上位をnum_words個表示
for topic_num in range(lda.num_topics):
    word_weight_dict_by_topic = lda.show_topic(topic_num, num_words)  # トピックごとの単語と重みのリスト
    print(f"Topic {topic_num}:  {word_weight_dict_by_topic}")  

Topic words
----------------------------------------------------
Topic 0:  [('店', 0.011591714), ('企業', 0.008040989), ('会社', 0.0049229898), ('年度', 0.004675564), ('客', 0.0046591647)]
Topic 1:  [('店', 0.012384828), ('会社', 0.006633756), ('市場', 0.0059722182), ('株', 0.0055713216), ('所', 0.0055485596)]
Topic 2:  [('企業', 0.0074932273), ('事業', 0.006691464), ('億', 0.0057583735), ('年度', 0.0052382345), ('店', 0.0046010492)]


Topic words
----------------------------------------------------
Topic 0:  [('万', 0.010659532), ('店', 0.007097314), ('市場', 0.0060652704), ('株', 0.005977563), ('億', 0.0057634674)]
Topic 1:  [('店', 0.0136192795), ('価格', 0.013214859), ('池袋', 0.007137636), ('られる', 0.005940716), ('コロナ', 0.0053854324)]
Topic 2:  [('価格', 0.018345227), ('企業', 0.009435166), ('店', 0.0069988365), ('万', 0.005895067), ('上昇', 0.005578275)]


Topic words
----------------------------------------------------
Topic 0:  [('価格', 0.019036062), ('企業', 0.006190807), ('所', 0.0053970544), ('ため', 0.005045468), ('値上げ', 0.004971116)]
Topic 1:  [('店', 0.01792086), ('価格', 0.009239445), ('人', 0.0057375603), ('取材', 0.005441921), ('客', 0.0054011364)]
Topic 2:  [('価格', 0.008679735), ('企業', 0.0076713343), ('証券', 0.007022484), ('会社', 0.0066977115), ('店', 0.0054056766)]


文書が少ないのでほぼ同じになっている
topic0:株への影響　topic2:実店舗への影響　2会社単位への影響　に大きく分けれる

##テストデータでやってみる

In [112]:
tests=["1月24日、ついにレギュラーガソリンの全国平均小売価格が170円を超えた。「長いコロナ不況で苦しいのに、ガソリン価格だけは右肩上がり。もう、ただのサラリーマンでは自家用車なんて持てないのかな」──そんな嘆きが、あちこちで聞かれるようになってきた。自動車業界は、燃料費の高騰だけが問題ではない。2年間も続いた未曾有の新型コロナウイルス禍。2022年こそは、ここまで控えられた分の“リベンジ消費”に期待がかかっていたが、蓋を開けてみると、買えるものも買えない状況に陥っている。都内のあるトヨタ系自動車販売店の営業マンは「コロナの世界的流行によって、半導体を生産する工場が、操業を何度も停止。部品供給が滞って、自動車工場もやむなく減産。去年8月に14年ぶりに全面改良したランドクルーザー（300系）も、問い合わせや注文は殺到しているのですが、納期は約4年後とお答えしているところなのです」と、肩を落としながら明かした。あるモータージャーナリストは「もともとランドクルーザーのセールスは、ほとんど海外。全生産の半分は中東、ロシアと豪州でも40％を占めて、日本の割合はわずか数％。供給台数が少ないので、さらに納期が遅れるのです」と補足した。さまざまな理由が重なって、新車の供給量で苦戦しているというのだ。また、高級車の代名詞でもあるメルセデス・ベンツの価格も上がってきている。かつて、400万円台から手を出すことのできたベンツCクラスは、「サラリーマンでも頑張ればベンツに乗れる象徴」として、多くのメルセデスユーザーを生んできた。しかし、昨年7年ぶりにフルモデルチェンジしたCクラスは、最低価格でも650万円以上。150万円以上の値上がりとなった。日本人の平均年収が約430万円（国税庁の「令和2年分　民間給与実態調査統計」より）だということを考えると、もはや「サラリーマンでは手が出せない超高級車」になってしまった。「しかも、そのベンツCクラスは半導体の不足により、当初予定していた装備が一部変更になっています。今後発売のロットは、高級スピーカーのブルメスターはオプションでも装着不可となり、トランクの下に足を入れると自動で開く『フットトランクオープナー』が付かないモデルもあります。しかもこれから新規発注すると納期は半年以上先とか……。AクラスやBクラスと比べて高級感があり、『ベビーSクラス』や『コンパクトSクラス』と言われていたCクラス。これだけ高額になったうえ、この仕様変更で購入を迷う人もいるかもしれませんね」（前出・モータージャーナリスト）もちろん、これらの変更はコロナ禍においては致し方ないこと。むしろ、半導体不足の中で、各自動車メーカーは新車を発売するために全力を注いでいるとは言える。一方で、車を購入予定の人の中には「新車が値上がりしたり納期に時間がかかる今、中古車はどうなの？」という気持ちになる人もいるかもしれない。だが、新車の供給不足の反動から、中古車市場は需要が高まり、値段も高騰している。",
      "大幅続伸。先週末に発表した自己株式の取得実施が買い材料視されている。発行済み株式数の4.23％に当たる300万株、10億円を取得上限としており、取得期間は2月21日から7月21日まで。機動的な資本政策の遂行及び株主への一層の利益還元を図ることを目的としている。当面の需給面での下支え材料につながるとの期待が先行へ。同社の自社株買いは、昨年10月にかけて上限10億円を実施して以来となる。"]

In [84]:
#どのトピックに分類されているか確認
def check_topic(test_texts,document,topic):
    test_keywords=get_keywords_list(test_texts)
    corpus_test = [dic.doc2bow(key) for key in test_keywords]
    pred = lda.inference(corpus_test)
    total=0
    for i in pred[0][document]:
        total += i
    return(pred[0][document][topic]/total)

In [111]:
check_topic(tests,0,2) #最初の文書のトピックは50%でtopic2 : 企業の生産の話
check_topic(tests,1,0) #2つ目の文書は67％でトピック0　株の話

0.01152370595030533

###備忘録

In [177]:
# test_keywords=get_keywords_list(tests)

In [ ]:
corpus_test = [dic.doc2bow(key) for key in test_keywords] #存在してなかったのは排除されている
# corpus_test = [dic.doc2bow(key, allow_update=True, return_missing=True) for key in keywords] #存在してなかったのを辞書に加えて更新

In [ ]:
pred = lda.inference(corpus_test)
# print(pred) #各トピックのweightを表示している

In [259]:
total=0
for i in pred[0][0]:
    total += i
pred[0][0][2]/total #全体のweigtの合計で割って％表示。　トピック3は73%とわかる

NameError: ignored

In [ ]:
total=0
for i in pred[0][1]:
    total += i
pred[0][1][0]/total 

#英語のfreesetを用いてやってみる

In [ ]:
https://newsdata.io/files/datasets/latest-news

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget -P /content/drive/MyDrive/SOCI https://newsdata.io/files/datasets/latest-news

--2022-02-21 13:25:36--  https://newsdata.io/files/datasets/latest-news
Resolving newsdata.io (newsdata.io)... 157.245.122.35
Connecting to newsdata.io (newsdata.io)|157.245.122.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 159412320 (152M) [application/zip]
Saving to: ‘/content/drive/MyDrive/SOCI/latest-news’

latest-news         100%[===================>] 152.03M  11.4MB/s    in 14s     

2022-02-21 13:25:51 (10.7 MB/s) - ‘/content/drive/MyDrive/SOCI/latest-news’ saved [159412320/159412320]



In [ ]:
!cp /content/drive/MyDrive/SOCI/latest-news .
!unzip latest-news

Archive:  latest-news
replace Latest_News/Latest_News.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

NameError: ignored

In [ ]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/Profmike/Latest_News.csv")
df["description"]

0        La Gazzetta dello Sport racconta del nervosism...
1        Il tecnico del Napoli è stato sanzionato per l...
2        Gli ultras del Napoli potrebbero rientrare all...
3        Decisivo nella costruzione del gioco e nella f...
4        Dopo il negativo inizio di stagione il preside...
                               ...                        
86555    Duminică, 24 octombrie, este o zi de sărbătoar...
86556    The group described Mr Tinubu as an exceptiona...
86557    Jakub Świerczok strzelił bramkę w meczu 33. ko...
86558    A woman in her 70s has died following a house ...
86559    “문재인 정부 성공과 정권 재창출 위해서 작은 힘이나마 보태겠습니다.”(이낙연 전 ...
Name: description, Length: 86560, dtype: object

In [ ]:
df

,title,link,keywords,creator,video_url,description,content,pubDate,full_description,image_url,source_id
0,Spalletti arrabbiato con i suoi calciatori nei...,https://www.areanapoli.it/rassegna-stampa/spal...,['Rassegna Stampa'],['AreaNapoli.it'],NaN,La Gazzetta dello Sport racconta del nervosism...,NaN,2021-10-26 07:06:10,NaN,https://cdn.areanapoli.it/immagini/notizie/big...,areanapoli
1,Squalifica e multa per Spalletti: la reazione ...,https://www.areanapoli.it/rassegna-stampa/squa...,['Rassegna Stampa'],['AreaNapoli.it'],NaN,Il tecnico del Napoli è stato sanzionato per l...,NaN,2021-10-26 07:06:10,NaN,https://cdn.areanapoli.it/immagini/notizie/big...,areanapoli
2,"Napoli-Bologna, gli ultras potrebbero entrare ...",https://www.areanapoli.it/rassegna-stampa/napo...,['Rassegna Stampa'],['AreaNapoli.it'],NaN,Gli ultras del Napoli potrebbero rientrare all...,NaN,2021-10-26 07:06:10,NaN,https://cdn.areanapoli.it/immagini/notizie/big...,areanapoli
3,"Juve, Dybala vale doppio: ecco perch&eacute; p...",https://www.tuttosport.com/news/calcio/serie-a...,['Juventus'],NaN,NaN,Decisivo nella costruzione del gioco e nella f...,NaN,2021-10-26 07:05:03,NaN,https://cdn.tuttosport.com/img/600/400/2021/10...,tuttosport
4,Tubertini non è più l'allenatore di Siena,https://www.tuttosport.com/news/pallavolo/a2-m...,"['Tubertini', 'Emma Villas']",NaN,NaN,Dopo il negativo inizio di stagione il preside...,NaN,2021-10-26 07:05:03,NaN,https://cdn.tuttosport.com/img/600/400/2021/10...,tuttosport
...,...,...,...,...,...,...,...,...,...,...,...
86555,Ziua cea mare a derbiurilor: Regal fotbalistic...,https://sport.hotnews.ro/stiri-fotbal-25130203...,['Sport/Fotbal'],NaN,NaN,"Duminică, 24 octombrie, este o zi de sărbătoar...",NaN,2021-10-24 08:51:00,"Duminică, 24 octombrie, este o zi de sărbătoar...",NaN,hotnews
86556,2023: Group mobilises support for Tinubu,https://www.premiumtimesng.com/news/more-news/...,"['More News', '2023 presidential election', 'A...",['Agency Report'],NaN,The group described Mr Tinubu as an exceptiona...,NaN,2021-10-24 08:50:59,"Umar Mohammed, Director-General, Tinubu Suppor...",NaN,premiumtimesng
86557,"Liga japońska. Jakub Świerczok z golem, remis ...",https://sport.interia.pl/pilka-nozna/news-liga...,['Piłka nożna'],NaN,NaN,Jakub Świerczok strzelił bramkę w meczu 33. ko...,NaN,2021-10-24 08:50:57,"Rozgrywki J1 League, najwyższego poziomu klubo...",https://i.iplsc.com/liga-japonska-jakub-swierc...,interiasport
86558,Woman (70s) dies following house fire in Cork ...,https://www.irishtimes.com/news/ireland/irish-...,NaN,NaN,NaN,A woman in her 70s has died following a house ...,NaN,2021-10-24 08:50:53,A woman in her 70s has died following a house ...,https://www.irishtimes.com/polopoly_fs/1.47092...,irishtimes
